In [12]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [13]:
#pkg = mp.jupyter.open_package()
pkg = doc = mp.jupyter.open_source_package()
pkg

# San Diego Police Calls for Service
`sandiego.gov-police_calls_for_service-1.1.1` Last Update: 2020-06-14T04:52:45

_San Diego Police Calls for Service, linked to related datasets_


This dataset links to the San Dieg Police Calls for Service datasets, from multiple years, combine the years and lints to the destriptions of disposition codes and beat neighborhoods. 

# Caveats and Adjustments

The meaning of the numbers in the `day_of_week` column has changed from earlier
to later years. Earlier years use values from 0 to 6, while later years use
vlaues from 1 to 7. Because of this, the `day_of_week` vlaues have been re-calculated to the span from 0 to 6, where 0 is Monday and 6 is Sunday.
## Documentation Links

* [Priority codes](http://seshat.datasd.org/pd/pd_cfs_priority_defs_datasd.pdf) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [sdpd_calls_dispositions](notebooks/Combine.ipynb#disp)**. San Diego Police Calls for Service, 2015 to 2020
* ** [sdpd_beats](notebooks/Combine.ipynb#beats)**. San Diego Police beats
* ** [sdpd_call_type](notebooks/Combine.ipynb#call_types)**. San Diego Police call type codes
* ** [sdpd_calls_for_service](notebooks/Combine.ipynb#df)**. San Diego Police Calls for Service, 2015 to 2020

## References
<ul><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2015_datasd_v1.csv">calls_for_service_source_2015</a></strong>. San Diego Police Calls for Service, 2015</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2016_datasd_v1.csv">calls_for_service_source_2016</a></strong>. San Diego Police Calls for Service, 2016</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2017_datasd_v1.csv">calls_for_service_source_2017</a></strong>. San Diego Police Calls for Service, 2017</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2018_datasd.csv">calls_for_service_source_2018</a></strong>. San Diego Police Calls for Service, 2018</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2019_datasd.csv">calls_for_service_source_2019</a></strong>. San Diego Police Calls for Service, 2019</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_calls_for_service_2020_datasd.csv">calls_for_service_source_2020</a></strong>. San Diego Police Calls for Service, 2020</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_dispo_codes_historical_datasd.csv">disp_codes_2017</a></strong>. Disposition Codes, prior to October 17, 2017</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv">disp_codes</a></strong>. Current disposition codes</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_beat_neighborhoods_datasd.csv">beat_codes</a></strong>. Beat Neighborhood Codes</li><li> <strong><a href="http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv">call_type_codes</a></strong>. Call type codes</li><ul>

In [14]:
frames = []

for r in pkg.references():
    
    if r.name.startswith('calls_for_service'):
        frames.append(r.dataframe())

df = pd.concat(frames)

df['date_time'] = pd.to_datetime(df.date_time)
df['day_of_week'] = df.date_time.dt.dayofweek

In [4]:
df.head().T

,0,1,2,3,4
incident_num,P15010000001,P15010000002,P15010000003,P15010000004,P15010000005
date_time,2015-01-01 00:00:14,2015-01-01 00:00:30,2015-01-01 00:00:48,2015-01-01 00:00:57,2015-01-01 00:01:05
day_of_week,3,3,3,3,3
address_number_primary,3800,0,3600,0,3300
address_dir_primary,NaN,NaN,NaN,NaN,NaN
address_road_primary,DELTA,60TH,LOGAN,33RD,43RD
address_sfx_primary,ST,ST,AV,ST,ST
address_dir_intersecting,NaN,NaN,NaN,NaN,NaN
address_road_intersecting,NaN,FEDERAL,NaN,IMPERIAL,NaN
address_sfx_intersecting,NaN,BL,NaN,AV,NaN


In [15]:
# Combine the disposition codes. It looks like there are some name changes, but mostly the laster 
# file is a superset of the earlier one
disp = pd.concat([pkg.reference('disp_codes').dataframe(), pkg.reference('disp_codes_2017').dataframe()]).sort_values('dispo_code')
disp = disp.drop_duplicates(subset=['dispo_code']).rename(columns={'dispo_code':'disposition'})


In [16]:
beats = pkg.reference('beat_codes').dataframe().rename(columns={'Beat':'beat'})

In [17]:
call_types = pkg.reference('call_type_codes').dataframe()
call_types = call_types[['call_type', 'description']] # Other collumns are empty
call_types

,call_type,description
0,1016,PRISONER IN CUSTODY
1,1016PT,PTU (PRISONER TRANSPORT)
2,1016QC,SHOPLIFTER/QUICK CITE
3,1021,PHONE YOUR STATION
4,1021H,PHONE YOUR HOME
...,...,...
284,TLO,TERRORISM LIAISON OFFICER
285,TP,TRAFFIC PURSUIT
286,TRO,"TRO VIOLATION, REPORT"
287,VARDA,VARDA ALARM
